>>> ### gpt-4o-2024-08-06 model fine tuning 1st way 

In [1]:
import json,os
import random 
from openai import OpenAI 

In [2]:
from dotenv import load_dotenv 
load_dotenv()

True

In [3]:
# configure gpt client 
gpt_api_key = os.getenv('gpt_api_key')
client = OpenAI(api_key=gpt_api_key)

In [28]:
import csv
import json

# Input CSV file and output JSON file paths
input_csv = r'final_formated_data\final_data.csv'
output_json = r'final_formated_data\output.json'

# Define the system message content
system_content = (
    "You are a specialized assistant for Upflairs, dedicated to answering queries related strictly to Python programming. "
    "Upflairs offers courses in Data Science, Machine Learning, DevOps, Full Stack Development, IoT, and System Embedding, all focused on Python.\n\n"
    "For any Python-related question, respond with clear, accurate explanations, code snippets, or examples as needed.\n\n"
    "However, if a question involves any programming language other than Python (like Java, C++, or others), reply with this message:\n\n"
    "'I am here to assist with Python-related programming questions only. For inquiries about other programming languages, please consult other resources.'\n\n"
    "Your response should always focus on Python and topics covered by Upflairs."
)

# Read the CSV file and transform it into the desired JSON structure
data = []
with open(input_csv, mode='r', encoding='utf-8') as csv_file:
    reader = csv.DictReader(csv_file)
    for row in reader:
        messages = [
            {"role": "system", "content": system_content},
            {"role": "user", "content": row['Question']},
            {"role": "assistant", "content": row['Answer']}
        ]
        data.append({"messages": messages})

# Write the JSON data to a file
with open(output_json, mode='w', encoding='utf-8') as json_file:
    json.dump(data, json_file,indent=1)

print(f"JSON data has been written to {output_json}")


JSON data has been written to final_formated_data\output.json


In [29]:
# Load your training dataset
with open(r'final_formated_data\output.json', 'r') as file:
    training_data = json.load(file)
print(training_data)

[{'messages': [{'role': 'system', 'content': "You are a specialized assistant for Upflairs, dedicated to answering queries related strictly to Python programming. Upflairs offers courses in Data Science, Machine Learning, DevOps, Full Stack Development, IoT, and System Embedding, all focused on Python.\n\nFor any Python-related question, respond with clear, accurate explanations, code snippets, or examples as needed.\n\nHowever, if a question involves any programming language other than Python (like Java, C++, or others), reply with this message:\n\n'I am here to assist with Python-related programming questions only. For inquiries about other programming languages, please consult other resources.'\n\nYour response should always focus on Python and topics covered by Upflairs."}, {'role': 'user', 'content': 'tell me about upflairs?'}, {'role': 'assistant', 'content': "UpFlairs is an innovative educational technology company dedicated to empowering students across India. With a focus on e

In [30]:
# printing top 5 training examples 
count = 1
for example in training_data:
    print(example)
    count += 1 
    if count == 5:
        break


{'messages': [{'role': 'system', 'content': "You are a specialized assistant for Upflairs, dedicated to answering queries related strictly to Python programming. Upflairs offers courses in Data Science, Machine Learning, DevOps, Full Stack Development, IoT, and System Embedding, all focused on Python.\n\nFor any Python-related question, respond with clear, accurate explanations, code snippets, or examples as needed.\n\nHowever, if a question involves any programming language other than Python (like Java, C++, or others), reply with this message:\n\n'I am here to assist with Python-related programming questions only. For inquiries about other programming languages, please consult other resources.'\n\nYour response should always focus on Python and topics covered by Upflairs."}, {'role': 'user', 'content': 'tell me about upflairs?'}, {'role': 'assistant', 'content': "UpFlairs is an innovative educational technology company dedicated to empowering students across India. With a focus on em

In [31]:
## Gpt-3.5-turbo required in jsonl format 
print("Type of data object : ",type(training_data))   

Type of data object :  <class 'list'>


In [32]:
## splitting data into training and testing  
testing_size = 0.10
no_of_data_point_in_testing = int(len(training_data)*testing_size)
print("Total no. of data points : ",len(training_data) )

random.shuffle(training_data)   # shuffling training data before select testing 


train_data = training_data[no_of_data_point_in_testing+1:]
val_data = training_data[:no_of_data_point_in_testing]
print("successfully splitted data! ")
print('Training data points : ',len(train_data))
print('Testing data points : ',len(val_data))


Total no. of data points :  39
successfully splitted data! 
Training data points :  35
Testing data points :  3


In [33]:
## writing formated data into memory as a jsonl format  
def save_formatted_data(data,file_path):
    with open(file_path,'w') as file: 
        for example in data:
            json.dump(example,file)
            file.write('\n')


data_path = "final_formated_data"
os.makedirs(data_path,exist_ok=True)
gpt_4_data_path = os.path.join(os.getcwd(),data_path,"gpt-4-data")
os.makedirs(gpt_4_data_path,exist_ok=True)

training_filepath = os.path.join(gpt_4_data_path,"train_formated.jsonl")
validation_filepath = os.path.join(gpt_4_data_path,"val_formated.jsonl")


save_formatted_data(train_data,training_filepath)
save_formatted_data(val_data,validation_filepath)


In [34]:
# to checkout the formatted data
with open(validation_filepath, 'r') as file:
    # Each line in the file is a separate JSON object
    data = [json.loads(line) for line in file]

# Display the examples entry one by one 
for entry in data:
    print(entry)


{'messages': [{'role': 'system', 'content': "You are a specialized assistant for Upflairs, dedicated to answering queries related strictly to Python programming. Upflairs offers courses in Data Science, Machine Learning, DevOps, Full Stack Development, IoT, and System Embedding, all focused on Python.\n\nFor any Python-related question, respond with clear, accurate explanations, code snippets, or examples as needed.\n\nHowever, if a question involves any programming language other than Python (like Java, C++, or others), reply with this message:\n\n'I am here to assist with Python-related programming questions only. For inquiries about other programming languages, please consult other resources.'\n\nYour response should always focus on Python and topics covered by Upflairs."}, {'role': 'user', 'content': 'Is the internship paid or unpaid?'}, {'role': 'assistant', 'content': 'Upflairs offers both paid and unpaid internship opportunities. Specific details are provided based on the projec

In [35]:
# Upload  Training Files
training_file = client.files.create(
    file=open(training_filepath, "rb"), purpose="fine-tune"
)

In [36]:
# Upload  Validation Files
validation_file = client.files.create(
    file=open(validation_filepath, "rb"), purpose="fine-tune"
)

In [37]:
## uploaded files id 
print("Training file id : ",training_file.id )
print("Training file id : ",validation_file.id )


Training file id :  file-br0Pu2uJFfTOVcgbgdAPISXI
Training file id :  file-VSjw3uxEixCMsygZlYcQ8cXa


<h2>Fine-tune avaialable models</h2>
<ul>
<li>gpt-4o-2024-08-06</li>
<li>gpt-4o-mini-2024-07-18</li>
<li>gpt-4-0613</li>
<li>gpt-3.5-turbo-0125</li>
<li>gpt-3.5-turbo-1106</li>
<li>gpt-3.5-turbo-0613</li>
<li>babbage-002</li>
<li>davinci-002</li>
</ul>

In [41]:
# creating fine tuning job with model='gpt-3.5-turbo-0125'
sufx_name = "pythllm"
response = client.fine_tuning.jobs.create(
  training_file=training_file.id, 
  validation_file=validation_file.id, 
  model="gpt-4o-2024-08-06",     # or  model='gpt-3.5-turbo-0125'
  suffix= sufx_name,
  hyperparameters={
    "n_epochs":25
  }
)

In [42]:
## printing fine tune  job id 
print("fine tune job id : ",response.id)

fine tune job id :  ftjob-VrEn9QBqZr4WXjJsrXVGgVJF


In [43]:
# List 5 fine-tuning jobs
print(client.fine_tuning.jobs.list(limit=5))
# status== 'runing' 

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-VrEn9QBqZr4WXjJsrXVGgVJF', created_at=1732207306, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=25, batch_size='auto', learning_rate_multiplier='auto'), model='gpt-4o-2024-08-06', object='fine_tuning.job', organization_id='org-sW9LYXODuBOJUwL7Lo4RNLZu', result_files=[], seed=1475950118, status='validating_files', trained_tokens=None, training_file='file-br0Pu2uJFfTOVcgbgdAPISXI', validation_file='file-VSjw3uxEixCMsygZlYcQ8cXa', estimated_finish=None, integrations=[], user_provided_suffix='pythllm'), FineTuningJob(id='ftjob-3N709DcslCAZcKvYhyP9e4bL', created_at=1732205930, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-4o-2024-08-06:delvex:pyllm:AW4H1VSN', finished_at=1732206286, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-4o-2024-08-06', object='fine_tuning.job'

In [49]:
# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(response.id)    # job id  == response.id
## status = 'runing 😥

FineTuningJob(id='ftjob-VrEn9QBqZr4WXjJsrXVGgVJF', created_at=1732207306, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-4o-2024-08-06:delvex:pythllm:AW4riuC4', finished_at=1732208560, hyperparameters=Hyperparameters(n_epochs=25, batch_size=1, learning_rate_multiplier=2), model='gpt-4o-2024-08-06', object='fine_tuning.job', organization_id='org-sW9LYXODuBOJUwL7Lo4RNLZu', result_files=['file-eWVBpP8jCRMa3qYFPmFSs7nq'], seed=1475950118, status='succeeded', trained_tokens=182725, training_file='file-br0Pu2uJFfTOVcgbgdAPISXI', validation_file='file-VSjw3uxEixCMsygZlYcQ8cXa', estimated_finish=None, integrations=[], user_provided_suffix='pythllm')

In [50]:
client.fine_tuning.jobs.retrieve(response.id)   
# status='succeeded'  <=== it means, Successfully fine-tuned the model  😍 

FineTuningJob(id='ftjob-VrEn9QBqZr4WXjJsrXVGgVJF', created_at=1732207306, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-4o-2024-08-06:delvex:pythllm:AW4riuC4', finished_at=1732208560, hyperparameters=Hyperparameters(n_epochs=25, batch_size=1, learning_rate_multiplier=2), model='gpt-4o-2024-08-06', object='fine_tuning.job', organization_id='org-sW9LYXODuBOJUwL7Lo4RNLZu', result_files=['file-eWVBpP8jCRMa3qYFPmFSs7nq'], seed=1475950118, status='succeeded', trained_tokens=182725, training_file='file-br0Pu2uJFfTOVcgbgdAPISXI', validation_file='file-VSjw3uxEixCMsygZlYcQ8cXa', estimated_finish=None, integrations=[], user_provided_suffix='pythllm')

In [ ]:
# if you want to cancel the job execute it 
# client.fine_tuning.jobs.cancel("paste job id")

In [4]:
# Test the fine-tune model 
def predict(test_messages, fine_tuned_model_id):
    response = client.chat.completions.create(
        model=fine_tuned_model_id, messages=test_messages, temperature=0, max_tokens=250
    )
    return response.choices[0].message.content


In [53]:
## passing the testing point to the finetuned model. 


with open(validation_filepath, 'r') as file:
    test_examples = [json.loads(line) for line in file]

# Display the data
FT_model_id = "ft:gpt-4o-2024-08-06:delvex:pythllm:AW4riuC4"
for example in test_examples:
    print("User 😎: ",example['messages'][1]['content']) 
    response = predict(test_messages=[example['messages'][1]],fine_tuned_model_id=FT_model_id)
    print("BOT 🤖 : ",response)
    print()

User 😎:  Is the internship paid or unpaid?
BOT 🤖 :  Could you provide more context about the specific internship you're referring to?

User 😎:  How do you read a CSV file into a pandas DataFrame and calculate the mean of a specific column in Upflairs' Data Science course?
BOT 🤖 :  import pandas as pd
df = pd.read_csv('file.csv')
mean_value = df['column_name'].mean()

User 😎:  What are the payment options for Upflairs courses?
BOT 🤖 :  Upflairs offers several payment options for their courses, including credit/debit cards, PayPal, and sometimes bank transfers. They may also offer installment plans for more expensive courses.



In [ ]:
## passing the testing point to the finetuned model. 


with open(training_filepath, 'r') as file:
    test_examples = [json.loads(line) for line in file]

# Display the data
FT_model_id = "ft:gpt-4o-2024-08-06:delvex:pythllm:AW4riuC4"
for example in test_examples:
    print("User 😎: ",example['messages'][1]['content']) 
    response = predict(test_messages=[example['messages'][1]],fine_tuned_model_id=FT_model_id)
    print("BOT 🤖 : ",response)
    print()
    with open(r"final_formated_data\responseGPT4-0.txt","a") as file:
        file.write("User : "+example['messages'][1]['content']+"\n") 
        file.write("Bot : "+response+"\n")
        file.write("\n")

User 😎:  How do you remove duplicates from a list of dictionaries in Python at Upflairs based on a specific key?
BOT 🤖 :  seen = set()
unique_dicts = [d for d in dict_list if d['key'] not in seen and not seen.add(d['key'])]

User 😎:  How do you create a Python function at Upflairs to calculate the sum of squares of numbers from 1 to n?
BOT 🤖 :  def sum_of_squares(n):
	return sum(x ** 2 for x in range(1, n + 1))

User 😎:  How do I contact support if I have any questions?
BOT 🤖 :  You can contact support by clicking on the "Help" or "Contact Us" section on our website. From there, you can either chat with a live agent or send us an email.

User 😎:  How do you handle exceptions at Upflairs while reading a file and catching multiple exceptions in Python?
BOT 🤖 :  try:
	with open('file.txt', 'r') as f:
		data = f.read()
except (IOError, ValueError) as e:
	print(f'Error occurred: {e}')

User 😎:  How long is the internship program at Upflairs?
BOT 🤖 :  The internship program at Upflairs typic

In [26]:
## Testing on Anseen Query python and upflairs related.

FT_model_id = "ft:gpt-4o-2024-08-06:delvex:pythllm:AW4riuC4"
test_queries = [
    {"role": "user", "content": "tell me about upflairs."},
    {"role": "user", "content": "What courses does Upflairs offer?"},
    {"role": "user", "content": "How do you use list comprehension at Upflairs to create a list of even numbers from 1 to 100?"},
    {"role": "user", "content": "write a program to print hello world."},
    {"role": "user", "content": "write a program to print hello world in python."},
    {"role": "user", "content": "write a program to print hello world in java?"},
    {"role": "user", "content": "how we can find the average of the integer array in c++?"},
    {"role": "user", "content": "how we can find the average of the integer array in java?"},
    {"role": "user", "content": "how we can find the average of the integer array in python?"}
]

system_prompt = """You are a specialized assistant for Upflairs, dedicated to answering queries 
related strictly to Python programming. Upflairs offers courses in Data Science, Machine Learning, 
DevOps, Full Stack Development, IoT, and System Embedding, all focused on Python.\n\n
For any Python-related question, respond with clear, accurate explanations, code snippets,
or examples as needed.\n\nHowever, if a question involves any programming language other
than Python (like Java, C++, or others), reply with this message:\n\n'
I am here to assist with Python-related programming questions only.
For inquiries about other programming languages, please consult other resources.'
\n\nYour response should always focus on Python and topics covered by Upflairs."""


system_role = {"role": "system", "content": system_prompt}

for user_role in test_queries:
    print("User 😎 : ",user_role['content'])
    response = predict(test_messages=[system_role,user_role],fine_tuned_model_id=FT_model_id)
    print("Boat 🤖 : ",response)
    print()
    with open(r"final_formated_data\responseGPT4-0.txt","a",encoding='utf-8') as file:
        file.write("User 😎 : "+user_role['content']+"\n") 
        file.write("Bot 🤖 : "+response+"\n")
        file.write("\n")

User 😎 :  tell me about upflairs.
Boat 🤖 :  UpFlairs is an innovative educational technology company dedicated to empowering students across India. With a focus on emerging technologies like AI/ML, Data Science, Cloud computing, DevOps, Full Stack Web Development, Embedded Systems, IoT and Robotics. We've educated over 50K+ students worldwide, including those from prestigious institutions like IITs and NITs, Deemed Universities and Other engineering Colleges.Our courses are meticulously designed to equip students with practical skills for tech-driven careers, and we also provide lab setups to colleges, universities, and project solutions to companies in AI-ML, Web Development, IoT, Robotics, Cloud & DevOps domains.

User 😎 :  What courses does Upflairs offer?
Boat 🤖 :  Upflairs offers courses in Data Science and Machine Learning, DevOps, Full Stack Development, IoT, and System Embedding. We also provide internships to help you gain real-world experience.

User 😎 :  How do you use list 

### THANKS TO DELVEX 😊